<a href="https://colab.research.google.com/github/varunsonawane/Data_Viz/blob/master/Data_Viz_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
# import data

!kaggle datasets download bhuvaneshprasad/ipl-dataset-2008-2023-only-dataset-you-need
!kaggle datasets download patrickb1912/ipl-complete-dataset-20082020
!kaggle datasets download kalilurrahman/ipl-player-auction-dataset-from-start-to-now

Dataset URL: https://www.kaggle.com/datasets/bhuvaneshprasad/ipl-dataset-2008-2023-only-dataset-you-need
License(s): apache-2.0
Dataset URL: https://www.kaggle.com/datasets/patrickb1912/ipl-complete-dataset-20082020
License(s): DbCL-1.0
Dataset URL: https://www.kaggle.com/datasets/kalilurrahman/ipl-player-auction-dataset-from-start-to-now
License(s): CC0-1.0


In [4]:
# unzip

!unzip /content/ipl-dataset-2008-2023-only-dataset-you-need.zip
!unzip /content/ipl-complete-dataset-20082020.zip
!unzip /content/ipl-player-auction-dataset-from-start-to-now.zip

Archive:  /content/ipl-dataset-2008-2023-only-dataset-you-need.zip
  inflating: ipl_ball_by_ball_data.csv  
  inflating: ipl_batting_card.csv    
  inflating: ipl_bowling_card.csv    
  inflating: ipl_fow_card.csv        
  inflating: ipl_historical.csv      
  inflating: ipl_partnership_card.csv  
  inflating: ipl_players_info.csv    
Archive:  /content/ipl-complete-dataset-20082020.zip
  inflating: deliveries.csv          
  inflating: matches.csv             
Archive:  /content/ipl-player-auction-dataset-from-start-to-now.zip
  inflating: IPLPlayerAuctionData.csv  


In [5]:
auction_df = pd.read_csv('/content/ipl-player-auction-dataset-from-start-to-now.zip')
auction_df.head()

,Player,Role,Amount,Team,Year,Player Origin
0,Aaron Finch,Batsman,40000000,Sunrisers Hyderabad,2014.0,Overseas
1,Aaron Finch,Batsman,32000000,Mumbai Indians,2015.0,Overseas
2,Aaron Finch,Batsman,10000000,Gujarat Lions,2016.0,Overseas
3,Aaron Finch,Batsman,62000000,Kings XI Punjab,2018.0,Overseas
4,Aaron Finch,Batsman,44000000,Royal Challengers Bangalore,2020.0,Overseas


In [6]:
auction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 970 entries, 0 to 969
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Player         970 non-null    object 
 1   Role           970 non-null    object 
 2   Amount         970 non-null    int64  
 3   Team           970 non-null    object 
 4   Year           969 non-null    float64
 5   Player Origin  970 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 45.6+ KB


In [7]:
# Null values
print(auction_df.isnull().sum())
auction_df.dropna(inplace=True)

Player           0
Role             0
Amount           0
Team             0
Year             1
Player Origin    0
dtype: int64


In [8]:
auction_df['Year'] = pd.to_datetime(auction_df['Year'], format='%Y').dt.year

In [9]:
auction_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 969 entries, 0 to 969
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Player         969 non-null    object
 1   Role           969 non-null    object
 2   Amount         969 non-null    int64 
 3   Team           969 non-null    object
 4   Year           969 non-null    int32 
 5   Player Origin  969 non-null    object
dtypes: int32(1), int64(1), object(4)
memory usage: 49.2+ KB


In [10]:
!pip install plotly==5.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 52.4 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [11]:
import plotly.express as px
import pandas as pd

In [12]:
team_spending = auction_df.groupby(['Year', 'Team'])['Amount'].sum().reset_index()

In [13]:
fig = px.line(team_spending, x='Year', y='Amount', color='Team',
                 title='Team Spending Over the Years')
fig.show()

/usr/local/lib/python3.11/dist-packages/plotly/express/_core.py:1992: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


In [14]:
bowlers_df = auction_df[auction_df['Role'] == 'Bowler']
batsmen_df = auction_df[auction_df['Role'] == 'Batsman']

In [15]:
bowlers_spending = bowlers_df.groupby('Year')['Amount'].sum().reset_index()
batsmen_spending = batsmen_df.groupby('Year')['Amount'].sum().reset_index()

In [16]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=bowlers_spending['Year'], y=bowlers_spending['Amount'],
                         mode='lines', name='Bowlers'))
fig.add_trace(go.Scatter(x=batsmen_spending['Year'], y=batsmen_spending['Amount'],
                         mode='lines', name='Batsmen'))

fig.update_layout(title='Spending on Bowlers vs. Batsmen Over the Years',
                  xaxis_title='Year',
                  yaxis_title='Total Spending')

fig.show()

In [17]:
import altair as alt

grouped = auction_df.groupby(['Year', 'Role'])['Amount'].sum().reset_index()

chart = alt.Chart(grouped).mark_area(
    interpolate='basis'  # makes it wavy like an actual streamgraph
).encode(
    x='Year:O',
    y='Amount:Q',
    color='Role:N'
).properties(
    width=1000,
    height=600,
    title='Streamgraph of Amount by Role over Years'
)

chart.show()


alt.Chart(...)

In [18]:
!pip install bar_chart_race

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 3.5 MB/s eta 0:00:00


In [27]:
import pandas as pd
import bar_chart_race as bcr

# Step 1: Aggregate by Team
team_race = (
    auction_df
    .groupby(['Year', 'Team'])['Amount']
    .sum()
    .reset_index()
)

# Step 2: Pivot - FULL teams, don't filter yet
pivot_team = team_race.pivot(index='Year', columns='Team', values='Amount').fillna(0)

# Step 3: Now in bar_chart_race, set n_bars=4 (dynamic top 4 every year)
bcr.bar_chart_race(
    df=pivot_team,
    filename=None,
    n_bars=4,  # Pick top 4 teams PER YEAR
    steps_per_period=40,
    period_length=2000,
    title='Top 4 Spending Teams by Year',
    bar_size=.90,
    figsize=(6, 4),
    interpolate_period=True,
    period_label={'x': .95, 'y': .25, 'ha': 'right', 'va': 'center'},
    cmap='bold',
    fixed_order=False,  # Important: False to allow different teams each year!
    fixed_max=True,
    filter_column_colors=True,
)


/usr/local/lib/python3.11/dist-packages/bar_chart_race/_make_chart.py:286: UserWarning:

set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.

/usr/local/lib/python3.11/dist-packages/bar_chart_race/_make_chart.py:287: UserWarning:

set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.



In [20]:
# Group by Player and Year
player_year = (
    auction_df
    .groupby(['Player', 'Year'])['Amount']
    .sum()
    .reset_index()
)


In [21]:
# Sort for shift to work properly
player_year = player_year.sort_values(by=['Player', 'Year'])

# Calculate YOY difference and YOY % change
player_year['Amount_Diff'] = player_year.groupby('Player')['Amount'].diff()
player_year['Amount_Pct_Change'] = player_year.groupby('Player')['Amount'].pct_change()


In [22]:
# Wildest positive jump (%)
wildest_rise = player_year.sort_values('Amount_Pct_Change', ascending=False).head(10)

# Wildest crash (%)
wildest_crash = player_year.sort_values('Amount_Pct_Change').head(10)


In [23]:
import plotly.express as px

# Pick a player from the big movers
target_player = wildest_rise.iloc[3]['Player']

# Filter data
player_data = player_year[player_year['Player'] == target_player]

# Plot
fig = px.line(
    player_data,
    x='Year',
    y='Amount',
    markers=True,
    title=f'Valuation Over Years: {target_player}',
    labels={'Amount': 'Auction Amount'}
)
fig.show()


In [24]:
import pandas as pd
import plotly.express as px

# --- 1. Group and Summarize ---
player_year = (
    auction_df
    .groupby(['Player', 'Year'])['Amount']
    .sum()
    .reset_index()
)

# Sort properly
player_year = player_year.sort_values(by=['Player', 'Year'])

# Calculate year-over-year change
player_year['Amount_Diff'] = player_year.groupby('Player')['Amount'].diff()
player_year['Amount_Pct_Change'] = player_year.groupby('Player')['Amount'].pct_change()

# --- 2. Find top rockets and crashes ---
# Filter out players with only one year (no diff possible)
player_with_diff = player_year.dropna(subset=['Amount_Diff'])

# Top 5 Rockets 🚀
rockets = player_with_diff.sort_values('Amount_Diff', ascending=False).head(5)

# Top 5 Crashes 💥
crashes = player_with_diff.sort_values('Amount_Diff', ascending=True).head(5)

# --- 3. Plotting Function ---
def plot_player_valuation(player_list, title):
    fig = px.line(
        player_year[player_year['Player'].isin(player_list)],
        x='Year',
        y='Amount',
        color='Player',
        markers=True,
        line_shape='spline',
        title=title,
        labels={'Amount': 'Auction Amount (₹)'}
    )
    fig.update_traces(mode="lines+markers+text", textposition="top right")
    fig.update_layout(
        title_font_size=24,
        font=dict(size=14),
        legend_title_text='Player',
        hovermode='x unified',
        height=600,
        width=900
    )
    fig.show()

# --- 4. Plot Top Rockets 🚀 ---
plot_player_valuation(rockets['Player'].tolist(), '🚀 Top 5 Player Valuation Rockets')

# --- 5. Plot Top Crashes 💥 ---
plot_player_valuation(crashes['Player'].tolist(), '💥 Top 5 Player Valuation Crashes')


/usr/local/lib/python3.11/dist-packages/plotly/express/_core.py:1992: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



/usr/local/lib/python3.11/dist-packages/plotly/express/_core.py:1992: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [29]:
import plotly.graph_objects as go

def plot_player_trends(player_list, title):
    df = player_year_team[player_year_team['Player'].isin(player_list)]

    fig = go.Figure()

    for player in player_list:
        player_df = df[df['Player'] == player]
        fig.add_trace(go.Scatter(
            x=player_df['Year'],
            y=player_df['Amount'],
            mode='lines+markers',
            name=player,
            hovertemplate=(
                'Player: %{text}<br>'
                'Year: %{x}<br>'
                'Amount: ₹%{y}<br>'
                'Team: %{customdata}'
            ),
            text=[player]*len(player_df),
            customdata=player_df['Team'],
            line=dict(width=3, shape='spline'),  # 🟢 Smoother lines
            marker=dict(size=8)
        ))

    fig.update_layout(
        title=title,
        height=600,
        width=950,
        font=dict(size=14),
        hovermode='closest',  # 🟢 Only show hovered point info
        legend_title_text='Player'
    )

    fig.show()

# Plot the smoother versions
plot_player_trends(rockets['Player'].tolist(), '🚀 Top 5 Rockets (Smoothed)')
plot_player_trends(crashes['Player'].tolist(), '💥 Top 5 Crashes (Smoothed)')


# **ipl-dataset-2008-2023**

In [33]:
print(matches_df.columns)
print(deliveries_df.columns)

Index(['id', 'season', 'city', 'date', 'match_type', 'player_of_match',
       'venue', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner',
       'result', 'result_margin', 'target_runs', 'target_overs', 'super_over',
       'method', 'umpire1', 'umpire2'],
      dtype='object')
Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batter', 'bowler', 'non_striker', 'batsman_runs', 'extra_runs',
       'total_runs', 'extras_type', 'is_wicket', 'player_dismissed',
       'dismissal_kind', 'fielder'],
      dtype='object')


In [30]:
matches_df = pd.read_csv('matches.csv')
matches_df.head()

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,2007/08,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,166.0,20.0,N,NaN,SJ Davis,DJ Harper
4,335986,2007/08,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,111.0,20.0,N,NaN,BF Bowden,K Hariharan


In [32]:
deliveries_df = pd.read_csv('deliveries.csv')
deliveries_df.head()

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NaN,NaN,NaN
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,1,1,wides,0,NaN,NaN,NaN
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN


In [40]:
query = """
batter == 'PJ Cummins'
"""

deliveries_df.query(query)

# print(deliveries_df['batter'].unique())

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
115958,829765,2,Kolkata Knight Riders,Chennai Super Kings,16,6,PJ Cummins,A Nehra,RN ten Doeschate,0,1,1,legbyes,0,NaN,NaN,NaN
115959,829765,2,Kolkata Knight Riders,Chennai Super Kings,17,1,PJ Cummins,DJ Bravo,RN ten Doeschate,0,0,0,NaN,1,PJ Cummins,caught,RA Jadeja
137775,1082595,2,Delhi Daredevils,Royal Challengers Bangalore,14,6,PJ Cummins,YS Chahal,RR Pant,0,0,0,NaN,0,NaN,NaN,NaN
137778,1082595,2,Delhi Daredevils,Royal Challengers Bangalore,15,3,PJ Cummins,TS Mills,RR Pant,1,0,1,NaN,0,NaN,NaN,NaN
137780,1082595,2,Delhi Daredevils,Royal Challengers Bangalore,15,5,PJ Cummins,TS Mills,RR Pant,0,0,0,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260846,1426312,1,Sunrisers Hyderabad,Kolkata Knight Riders,17,3,PJ Cummins,SP Narine,JD Unadkat,2,0,2,NaN,0,NaN,NaN,NaN
260847,1426312,1,Sunrisers Hyderabad,Kolkata Knight Riders,17,4,PJ Cummins,SP Narine,JD Unadkat,1,0,1,NaN,0,NaN,NaN,NaN
260850,1426312,1,Sunrisers Hyderabad,Kolkata Knight Riders,18,1,PJ Cummins,AD Russell,B Kumar,0,0,0,NaN,0,NaN,NaN,NaN
260851,1426312,1,Sunrisers Hyderabad,Kolkata Knight Riders,18,2,PJ Cummins,AD Russell,B Kumar,0,0,0,NaN,0,NaN,NaN,NaN


In [41]:
# Merge to get season info
deliveries = deliveries_df.merge(matches_df[['id', 'season']], left_on='match_id', right_on='id', how='left')

# Batting stats
cummins_batting = deliveries[deliveries['batter'] == 'PJ Cummins']
batting_stats = cummins_batting.groupby('season').agg(
    runs_scored=('batsman_runs', 'sum'),
    balls_faced=('ball', 'count')
).reset_index()
batting_stats['strike_rate'] = (batting_stats['runs_scored'] / batting_stats['balls_faced']) * 100

# Bowling stats
cummins_bowling = deliveries[deliveries['bowler'] == 'PJ Cummins']
bowling_stats = cummins_bowling.groupby('season').agg(
    balls_bowled=('ball', 'count'),
    runs_conceded=('total_runs', 'sum'),
    wickets=('is_wicket', 'sum')
).reset_index()
bowling_stats['overs'] = bowling_stats['balls_bowled'] // 6 + (bowling_stats['balls_bowled'] % 6) / 10
bowling_stats['economy_rate'] = bowling_stats['runs_conceded'] / (bowling_stats['balls_bowled'] / 6)


In [42]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a subplot figure with 2 rows
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Batting: Runs & Strike Rate", "Bowling: Wickets & Economy Rate"),
    shared_xaxes=True,
    vertical_spacing=0.15
)

# Row 1: Batting
fig.add_trace(go.Bar(x=batting_stats['season'], y=batting_stats['runs_scored'], name="Runs Scored", marker_color='green'), row=1, col=1)
fig.add_trace(go.Scatter(x=batting_stats['season'], y=batting_stats['strike_rate'], name="Strike Rate", yaxis="y2", marker_color='black', mode='lines+markers'), row=1, col=1)

# Row 2: Bowling
fig.add_trace(go.Bar(x=bowling_stats['season'], y=bowling_stats['wickets'], name="Wickets", marker_color='blue'), row=2, col=1)
fig.add_trace(go.Scatter(x=bowling_stats['season'], y=bowling_stats['economy_rate'], name="Economy Rate", yaxis="y4", marker_color='red', mode='lines+markers'), row=2, col=1)

# Layout tweaks
fig.update_layout(
    height=700,
    title_text="PJ Cummins' Batting & Bowling Performance Over the Years",
    yaxis2=dict(overlaying='y', side='right', title='Strike Rate'),
    yaxis4=dict(overlaying='y3', side='right', title='Economy Rate'),
    hovermode='x unified',
    legend=dict(x=0.01, y=1.05, orientation="h")
)

fig.show()
